# Chapter 5 일급 함수
파이썬의 함수는 일급 객체다. 다음과 같은 작업을 수행할 수 있는 프로그램 개체를 일급 객체로 정의한다.
- 런타임에 생성할 수 있다.
- 데이터 구조체의 변수나 요소에 할당할 수 있다.
- 함수 인수로 전달할 수 있다.
- 함수 결과로 반환할 수 있다.

# 5.1 함수를 객체처럼 다루기
- 데이터 구조체의 변수나 요소에 할당할 수 있다.

In [1]:
def factorial(n):
    return 1 if n <2 else n * factorial(n-1)

fact = factorial
print(fact)
print(fact(5))

<function factorial at 0x000001C8251CD820>
120


# 5.2 고위 함수 ( Higher - order function )
- 함수로 인수로 받거나, 함수를 결과로 반환하는 함수를 고위 함수라고 한다.
- map(), filter(), reduce() 고위 함수는 여전히 존재하지만, 대부분의 경우 더 나은 방법이 있다.

## 5.2.1 map(), filter(), reduce() 의 대안
- map() 과 filter() 는 제너레이터 ( 일종의 반복 가능 객체 ) 를 반환하므로 제너레이터 표현식이 이 함수들을 직접 대체

In [2]:
print(list(map(fact, range(6))))
print([fact(n) for n in range(6)])
print(list(map(factorial , filter(lambda n : n%2 , range(6)))))
print([fact(n) for n in range(6) if n % 2  ])

[1, 1, 2, 6, 24, 120]
[1, 1, 2, 6, 24, 120]
[1, 6, 120]
[1, 6, 120]


# 5.3 익명 함수 ( Lambda )
- lambda 키워드는 파이썬 표현식 내에 익명 함수를 생성
- lambda 본체에서는 할당문이나 while, try 등 파이썬 문장 사용 불가
- 고위 함수의 인수로 사용하는 방법 외에 lambda 함수는 파이썬에서 거의 사용하지 않는다.
    - 구문 제한성 & 가독성 저하
- refactoring is the process of reconstructuring code, whilde not chaning its original functionality
- def 문과 마찬가지로 lambda 표현식도 하나의 함수 객체를 만든다.

In [3]:
fruits = ['strawbery','fig','apple','cherry','raspberry','banana']
print(sorted(fruits, key = lambda word : word[::-1]))

['banana', 'apple', 'fig', 'strawbery', 'raspberry', 'cherry']


# 5.4 일곱 가지 맛의 콜러블 객체
- 호출할 수 있는 객체인지 알아보려면 callable() 내장 함수를 이용한다.
    - 사용자 정의 함수 / 내장 함수 / 내장 메서드 / 메서드 / 클래스 / 클래스 객체 / 제너레이터 함수

In [4]:
abs, str, 13

(<function abs(x, /)>, str, 13)

In [5]:
[callable(obj) for obj in (abs, str, 13)]

[True, True, False]

# 5.5 사용자 정의 콜러블형
- 파이썬 함수가 실제 객체일 뿐만 아니라, 모든 파이썬 객체가 함수처럼 동작하게 만들 수 있다.
- \_\_call__() method 를 통해 bingo 객체를 함수처럼 호출할 때 ( e.g. bingo() ) 어떻게 작동되는지 정할 수 있음. 여기서는 객체를 함수처럼 호출하면 pick() method가 실행됨

In [6]:
import random 

class BingoCage:
    
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)
        
    def pick(self):
        try :
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        return self.pick()

In [7]:
bingo = BingoCage(range(3))
bingo.pick()

2

In [8]:
bingo()

0

In [9]:
callable(bingo)

True

# 5.6 함수 인트로스펙션
- 일반적인 사용자 정의 클래스의 객체가 마찬가지로 함수는 \_\_dict__ 속성을 이용해서 객체에 할당된 사용자 속성을 보관한다.
    - 이 속성은 기본적인 애너테이션 형태로서 쓸모가 많다.
- 일반적으로 어떤 속성을 함수에 할당하는 것은 흔한 일이 아니나 사용 가능하긴함. 아래의 upper_case_name이 그 예시

In [10]:
print(dir(factorial))

['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [11]:
def upper_case_name(obj):
    return ("%s %s" % (obj.first_name, obj_last_name)).upper()

upper_case_name.short_description = 'Customer name'

In [12]:
print(upper_case_name.__dict__)

{'short_description': 'Customer name'}


In [13]:
class C: pass
obj = C()
def func(): pass
print(sorted(set(dir(func)) - set(dir(obj)) ))

['__annotations__', '__call__', '__closure__', '__code__', '__defaults__', '__get__', '__globals__', '__kwdefaults__', '__name__', '__qualname__']


# 5.7 위치 매개변수에서 키워드 전용 매개변수까지
- 키워드 전용 매개 변수 ( keyword only argument ) 란 키워드 인수로만 전달될 수 있으며 결코 익명의 위치 인수로는 전달되지 않는 인수를 말한다.
- 키워드 전용 인수를 지정하려면 * 가 붙은 인수 뒤에 이름을 지정한다.
- 키워드 전용 인수는 기본 값을 지정하지 않아도 되며, 이후 나올 f 함수의 b 인수를 필수 인수로 만든다.

In [14]:
def tag(name, *content, cls=None, **attrs):
    """하나 이상의 HTML 태그를 생성"""
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value)
                          for attr, value in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' %
                        (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

In [15]:
print(tag('br'))
print(tag('p','hello'))
print(tag('p','hello','world'))
print(tag('p','hello',id=33))
print(tag('p', 'hello', 'world', cls='sidebar'))
print(tag(content='testing', name='img'))


my_tag = {'name':'img', 'title':'Sunset Boulevard',
         'src':'sunset.jpg', 'cls':'framed'}

print(tag(**my_tag))

<br />
<p>hello</p>
<p>hello</p>
<p>world</p>
<p id="33">hello</p>
<p class="sidebar">hello</p>
<p class="sidebar">world</p>
<img content="testing" />
<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />


In [16]:
def f(a, *c, b):
    return a,b
print(f(1 ,2,3,4,5,6, b = 2))

(1, 2)


# 5.8 매개변수에 대한 정보 읽기
- 함수 객체 안의 \_\_defaults__ 속성에는 위치 인수와 키워드 인수의 기본 값을 가진 튜플이 들어있다.
- 키워드 전용 인수의 기본 값은 \_\_kwdefaults__ 속성에 들어가 있다.
- 인수명은 \_\_code__ 속성에 들어있는데, 이 속성은 여러 속성을 담고 있는 code 객체를 가르킨다.

In [17]:
def clip(text, max_len = 80):
    """max_len 앞이나 뒤의 마지막 공백에서 잘라낸 텍스르를 반환한다.
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0 ,max_len )
        if space_before >= 0 :
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:
        end = len(text)
    return text[:end].rstrip()

In [18]:
from clip import clip
print(clip.__defaults__)
print(clip.__code__)
print(clip.__code__.co_varnames)
print(clip.__code__.co_argcount)

# 정보를 보기 쉽지 않음 => inspect module 이용

(80,)
<code object clip at 0x000001C8251FCB30, file "C:\Users\user\Desktop\Python\FluentPython\MunKi\clip.py", line 2>
('text', 'max_len', 'end', 'space_before', 'space_after')
2


In [19]:
from clip import clip
from inspect import signature
sig = signature(clip)
print(sig)

(text, max_len=80)


In [20]:
print(str(sig))

(text, max_len=80)


In [21]:
for name, param in sig.parameters.items():
    print(param.kind , ' : ' , name , ' = ' , param.default)

POSITIONAL_OR_KEYWORD  :  text  =  <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD  :  max_len  =  80


In [22]:
import inspect
sig = inspect.signature(tag)
my_tag = {'name':'img', 'title':'Sunset Boulevard',
         'src':'sunset.jpg', 'cls':'framed'}
bound_args = sig.bind(**my_tag)
print(bound_args)

<BoundArguments (name='img', cls='framed', attrs={'title': 'Sunset Boulevard', 'src': 'sunset.jpg'})>


In [23]:
for name, value in bound_args.arguments.items(): # arguments 는 OrderedDict 형
    print(name , ' = ' , value)

name  =  img
cls  =  framed
attrs  =  {'title': 'Sunset Boulevard', 'src': 'sunset.jpg'}


In [24]:
del my_tag['name']
bound_args = sig.bind(**my_tag) # name 빼고 binding 하면 name 이 빠져있다고 나옴

TypeError: missing a required argument: 'name'

# 5.9 함수 애너테이션
- 파이썬 3는 함수의 매개변수와 반환 값에 메타데이터를 추가할 수 있는 구문을 제공한다.
- 함수 선언에서 각 매개변수에는 콜론(:) 뒤에 애너테이션 표현식을 추가할 수 있다.
    - 기본 값이 있을 때, 에너테이션은 인수명과 등호(=)사이에 들어간다.
- 반환 값에 애너테이션을 추가하려면 매개변수를 닫는 괄호와 함수 선언의 제일 뒤에 오는 콜론 사이에 -> 기호와 표현식을 추가한다.
- 표현식은 어떤 자료형도 될 수 있다.
    - str, int 와 같은 클래스, 'int>0' 와 같은 문자열이 애너테이션에서 가장 널리 사용되는 자료형이다.

In [25]:
def clip(text:str, max_len:'int>0' = 80) -> str :
    """max_len 앞이나 뒤의 마지막 공백에서 잘라낸 텍스르를 반환한다.
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0 ,max_len )
        if space_before >= 0 :
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:
        end = len(text)
    return text[:end].rstrip()

In [27]:
#from clip import clip_annot
#print(clip_annot.__annotations__)

print(clip.__annotations__)

{'text': <class 'str'>, 'max_len': 'int>0', 'return': <class 'str'>}


In [28]:
# 다음과 같은 annotation 으로 list[str] 로 type 이 list 이고 그 list 의 요소가 str 인것을 알릴 수도 있음. 
def clip_annot(text:list, max_len:'int>0' = 80) -> list[str] :
    """max_len 앞이나 뒤의 마지막 공백에서 잘라낸 텍스르를 반환한다.
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0 ,max_len )
        if space_before >= 0 :
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:
        end = len(text)
    return text[:end].rstrip().split()

In [30]:
print(clip_annot.__annotations__)

{'text': <class 'list'>, 'max_len': 'int>0', 'return': list[str]}


In [31]:
from clip import clip_annot
from inspect import signature
sig = signature(clip_annot)
sig.return_annotation

str

In [32]:
for param in sig.parameters.values():
    note = repr(param.annotation).ljust(13)
    print(note, ':', param.name, '=' , param.default)

<class 'str'> : text = <class 'inspect._empty'>
'int>0'       : max_len = 80


In [33]:
sig.parameters

mappingproxy({'text': <Parameter "text: str">,
              'max_len': <Parameter "max_len: 'int>0' = 80">})

In [38]:
sig.parameters['text'].annotation

str

# 5.10 함수형 프로그래밍을 위한 패키지

## 5.10.1 operator 모듈
- operator 모듈은 시퀀스에서 항목을 가져오는 람다를 대체하는 itemgetter() 함수와 객체의 속성을 읽는 람다를 대체하는 atrgetter() 함수를 제공한다.
- itemgetter()
    - itemgetter(1) 은 본질적으로 lambda x :x[1]과 동일하다.
    - itemgetter()에 여러 개의 인덱스를 인수로 전달하면, 생성된 함수는 해당 인덱스의 값들로 구성된 튜플을 반환한다.
    - itemgetter() 는 [] 연산자를 사용하므로 시퀀스뿐만 아니라 매핑 및 \_\_getitem__() 을 구현한 모든 클래스를 지원한다.
- attrgetter()
    - itemgetter() 의 형제인 attrgetter() 는 이름으로 객체 속성을 추출하는 함수를 생성한다.
    - attrgetter() 에 여러 속성명을 인수로 전달하면, 역시 해당 속성 값으로 구성된 튜플을 반환한다.
    - 속성명에 점(.)이 포함되어 있으면 attrgetter()는 내포된 객체를 찾아서 해당 속성을 가져온다.
- methodcaller()  # operator 의 함수
    - methodcaller() 는 실행 중 함수를 생성한다는 점에서 attrgetter() 나 itemgetter() 메서드와 비슷하다.
    - methodcaller() 가 생성한 함수는 인수로 전달받은 객체의 메서드를 호출한다.

In [39]:
from functools import reduce

def fact(n):
    return reduce( lambda a, b: a*b , range(1,n+1))

In [40]:
from functools import reduce
from operator import mul

def fact(n):
    return reduce( mul, range(1,n+1) )

In [41]:
metro_data = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),   # <1>
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

In [42]:
from operator import itemgetter
for city in sorted(metro_data, key = itemgetter(1)):
    print(city)

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))


In [43]:
cc_name = itemgetter(1,0)
for city in metro_data:
    print(cc_name(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


In [44]:
from operator import itemgetter
for city in sorted(metro_data, key = itemgetter(1)):
    print(city)

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))


In [45]:
from collections import namedtuple
LatLong = namedtuple('LatLong', 'lat long')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')
metro_areas = [Metropolis(name, cc, pop, LatLong(lat, long))
              for name, cc, pop, (lat, long) in metro_data]

In [46]:
print(metro_areas[0])

Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLong(lat=35.689722, long=139.691667))


In [47]:
print(metro_areas[0].coord.lat)

35.689722


In [48]:
from operator import attrgetter
name_lat = attrgetter('name','coord.lat')

for city in sorted(metro_areas, key = attrgetter('coord.lat')):
    print(name_lat(city))

('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


In [64]:
name_lat

operator.attrgetter('name', 'coord.lat')

In [49]:
import operator
[name for name in dir(operator) if not name.startswith('_')]

['abs',
 'add',
 'and_',
 'attrgetter',
 'concat',
 'contains',
 'countOf',
 'delitem',
 'eq',
 'floordiv',
 'ge',
 'getitem',
 'gt',
 'iadd',
 'iand',
 'iconcat',
 'ifloordiv',
 'ilshift',
 'imatmul',
 'imod',
 'imul',
 'index',
 'indexOf',
 'inv',
 'invert',
 'ior',
 'ipow',
 'irshift',
 'is_',
 'is_not',
 'isub',
 'itemgetter',
 'itruediv',
 'ixor',
 'le',
 'length_hint',
 'lshift',
 'lt',
 'matmul',
 'methodcaller',
 'mod',
 'mul',
 'ne',
 'neg',
 'not_',
 'or_',
 'pos',
 'pow',
 'rshift',
 'setitem',
 'sub',
 'truediv',
 'truth',
 'xor']

In [50]:
from operator import methodcaller
s = 'The time has come'
upcase = methodcaller('upper')
print(upcase(s))

THE TIME HAS COME


In [51]:
hiphenate = methodcaller('replace', ' ', '-')
print(hiphenate(s))

The-time-has-come


In [65]:
str.upper(s)

'THE TIME HAS COME'

## 5.10.2 functools.partial()로 인수 고정하기
- functools 모듈은 몇 가지 고위 함수를 통합한다. 그중 가장 널리 알려진 함수가 reduce()함수다.
- functools에서 제공하는 나머지 함수 중 partial() 및 이의 변형인 partialmethod() 함수가 매우 유용
- functools.partial() 은 함수를 부분적으로 실행할 수 있게 해주는 고위함수다.
- partial() 의 첫 번째 인수는 콜러블이며, 그 뒤에 바인딩할 위치 인수와 키워드 인수가 원하는 만큼 나온다.

In [53]:
from operator import mul
from functools import partial
triple = partial(mul, 3)
print(triple(7))

21


In [54]:
print(list(map(triple, range(1,10))))

[3, 6, 9, 12, 15, 18, 21, 24, 27]


In [55]:
# mul() 함수는 이 예제의 map() 안에서 사용할 수 없다. ( 이 경우는 고정되어 있는 인수가 없기 때문에)
print(list(map( mul , range(1,10))))

TypeError: mul expected 2 arguments, got 1

In [56]:
import unicodedata, functools
nfc = functools.partial(unicodedata.normalize, 'NFC')
s1 = 'café'
s2 = 'cafe\u0301'
print(s1, s2)

café café


In [57]:
print(s1 == s2)

False


In [58]:
print(nfc(s1) == nfc(s2))

True


In [59]:
from functools import partial
picture = partial(tag,'img', cls = 'pic-frame')
print(picture(src='wumpus.jpeg'))

<img class="pic-frame" src="wumpus.jpeg" />


In [60]:
print(picture)

functools.partial(<function tag at 0x000001C8251CDF70>, 'img', cls='pic-frame')


In [66]:
print(picture.func)

<function tag at 0x000001C8251CDF70>


In [62]:
# functools.parial 객체는 원래 함수와 고정된 인수에 접근할 수 있는 속성을 가지고 있다.
print(picture.func)
print(picture.args)     # positional arguments
print(picture.keywords) # keyword only arguments

<function tag at 0x000001C8251CDF70>
('img',)
{'cls': 'pic-frame'}
